# Descriptive Statistics

## Data Cleaning

In [40]:
import pandas as pd
import numpy as np

import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [2]:
default = pd.read_csv('UCI_Credit_Card.csv')

In [3]:
# ID: ID of each client
# LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
# SEX: Gender (1=male, 2=female) -> replace by 0 and 1 - 'Female = 0'
# EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown)
# MARRIAGE: Marital status (1=married, 2=single, 3=others) -> oneHotEnconding
# AGE: Age in years
# PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
# PAY_2: Repayment status in August, 2005 (scale same as above)
# PAY_3: Repayment status in July, 2005 (scale same as above)
# PAY_4: Repayment status in June, 2005 (scale same as above)
# PAY_5: Repayment status in May, 2005 (scale same as above)
# PAY_6: Repayment status in April, 2005 (scale same as above)
# BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
# BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
# BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
# BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
# BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
# BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
# PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
# PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
# PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
# PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
# PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
# PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
# default.payment.next.month: Default payment (1=yes, 0=no)

In [4]:
default.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [5]:
default['PAY_4'].unique()

array([-1,  0, -2,  2,  3,  4,  5,  7,  6,  1,  8], dtype=int64)

In [6]:
default.rename(columns={'PAY_0':'PAY_1'}, inplace=True)

In [7]:
#pay duly converted to 0
default['PAY_1'] = np.where((default.PAY_1 == -1)|(default.PAY_1 == -2), 0, default.PAY_1)
default['PAY_2'] = np.where((default.PAY_2 == -1)|(default.PAY_2 == -2), 0, default.PAY_2)
default['PAY_3'] = np.where((default.PAY_3 == -1)|(default.PAY_3 == -2), 0, default.PAY_3)
default['PAY_4'] = np.where((default.PAY_4 == -1)|(default.PAY_4 == -2), 0, default.PAY_4)
default['PAY_5'] = np.where((default.PAY_5 == -1)|(default.PAY_5 == -2), 0, default.PAY_5)
default['PAY_6'] = np.where((default.PAY_6 == -1)|(default.PAY_6 == -2), 0, default.PAY_6)

In [8]:
#Checking unique values
pay_values = default[['PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']].values.ravel()
pd.unique(pay_values)

array([2, 0, 1, 3, 4, 5, 6, 8, 7], dtype=int64)

In [9]:
default.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_1                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [10]:
default['EDUCATION'].unique()

array([2, 1, 3, 5, 4, 6, 0], dtype=int64)

In [11]:
default['EDUCATION'] = np.where((default.EDUCATION == 0)|(default.EDUCATION == 6), 5, default.EDUCATION)

In [12]:
default['MARRIAGE'] = np.where((default.MARRIAGE == 0),3, default.MARRIAGE)

In [13]:
default['MARRIAGE'] = np.where((default.MARRIAGE == 1),'married', default.MARRIAGE)
default['MARRIAGE'] = np.where((default.MARRIAGE == '2'),'single', default.MARRIAGE)
default['MARRIAGE'] = np.where((default.MARRIAGE == '3'),'other', default.MARRIAGE)

In [14]:
default["MARRIAGE"].value_counts()

single     15964
married    13659
other        377
Name: MARRIAGE, dtype: int64

In [15]:
default['SEX'].unique()

array([2, 1], dtype=int64)

In [16]:
default['SEX'] = np.where((default.SEX == 2),0, default.SEX)

In [17]:
default['AGE'].max()

79

In [18]:
default['AGE'].min()

21

In [19]:
def range_age_column(row):
    if row['AGE'] < 31:
        return '20-30'
    elif 31 <= row['AGE'] < 41:
        return '31-40'
    elif 41 <= row['AGE'] < 51:
        return '41-50'
    elif 51 <= row['AGE'] < 61:
        return '51-60'
    elif 60 <= row['AGE']:
        return '60+'

In [20]:
default['Range_age'] = default.apply(range_age_column, axis=1)

In [21]:
default['Range_age'].value_counts()

20-30    11013
31-40    10713
41-50     6005
51-60     1997
60+        272
Name: Range_age, dtype: int64

In [22]:
def ordinal_age (row):
    if row in range(20,31):
        return 1
    elif row in range(31,41):
        return 2
    elif row in range(41,51):
        return 3
    elif row in range(51,61):
        return 4
    else:
        return 5

In [23]:
default['ORD_AGE'] = default['AGE'].apply(ordinal_age)

In [24]:
default['ORD_AGE'].unique()

array([1, 2, 4, 3, 5], dtype=int64)

In [25]:
default.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month', 'Range_age', 'ORD_AGE'],
      dtype='object')

In [26]:
default.drop(columns='ID', inplace=True)

In [27]:
#default.to_csv('data_model.csv', index=False)

In [28]:
# Percentage of men
prc_men = len(default[default['SEX']==1])/len(default)
prc_men

0.39626666666666666

In [29]:
# being in default
nr_defaults = len(default[default['default.payment.next.month']==1])
print('nr defaults =', nr_defaults)
print('% of defaults =', nr_defaults/len(default))

nr defaults = 6636
% of defaults = 0.2212


In [30]:
default['EDUCATION'].value_counts()

2    14030
1    10585
3     4917
5      345
4      123
Name: EDUCATION, dtype: int64

In [31]:
default['MARRIAGE'].value_counts()

single     15964
married    13659
other        377
Name: MARRIAGE, dtype: int64

In [32]:
range_ages = default.groupby("Range_age")["default.payment.next.month"].value_counts()
range_ages

Range_age  default.payment.next.month
20-30      0                             8542
           1                             2471
31-40      0                             8524
           1                             2189
41-50      0                             4606
           1                             1399
51-60      0                             1493
           1                              504
60+        0                              199
           1                               73
Name: default.payment.next.month, dtype: int64

In [33]:
range_age = list(default['Range_age'].unique())


In [42]:
default['LIMIT_BAL'].unique()

array([  20000.,  120000.,   90000.,   50000.,  500000.,  100000.,
        140000.,  200000.,  260000.,  630000.,   70000.,  250000.,
        320000.,  360000.,  180000.,  130000.,  450000.,   60000.,
        230000.,  160000.,  280000.,   10000.,   40000.,  210000.,
        150000.,  380000.,  310000.,  400000.,   80000.,  290000.,
        340000.,  300000.,   30000.,  240000.,  470000.,  480000.,
        350000.,  330000.,  110000.,  420000.,  170000.,  370000.,
        270000.,  220000.,  190000.,  510000.,  460000.,  440000.,
        410000.,  490000.,  390000.,  580000.,  600000.,  620000.,
        610000.,  700000.,  670000.,  680000.,  430000.,  550000.,
        540000., 1000000.,  530000.,  710000.,  560000.,  520000.,
        750000.,  640000.,   16000.,  570000.,  590000.,  660000.,
        720000.,  327680.,  740000.,  800000.,  760000.,  690000.,
        650000.,  780000.,  730000.])

In [73]:
default['LIMIT_BAL'].median()

140000.0

In [74]:
default['LIMIT_BAL'].mean()

167484.32266666667

In [34]:
# SEX: Gender (1=male, 0=female)
# EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown)
# MARRIAGE: Marital status (1=married, 2=single, 3=others)

In [36]:
#verify different groups (age+marriage+studies+gender)

for g in [0,1]:
    for m in ['married','single','others']:
        for e in [1,2,3,4,5]:
            for a in range_age:
                try:
                    print (round(
                        len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                (default['SEX']==g) & (default['default.payment.next.month']==1)]) /
                        len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                 (default['SEX']==g)]),2), '    code=',a,e,m,g)
                except:
                    continue

0.21     code= 20-30 1 married 0
0.18     code= 31-40 1 married 0
0.21     code= 51-60 1 married 0
0.17     code= 41-50 1 married 0
0.32     code= 60+ 1 married 0
0.27     code= 20-30 2 married 0
0.21     code= 31-40 2 married 0
0.27     code= 51-60 2 married 0
0.23     code= 41-50 2 married 0
0.28     code= 60+ 2 married 0
0.27     code= 20-30 3 married 0
0.22     code= 31-40 3 married 0
0.24     code= 51-60 3 married 0
0.27     code= 41-50 3 married 0
0.22     code= 60+ 3 married 0
0.0     code= 20-30 4 married 0
0.0     code= 31-40 4 married 0
0.0     code= 51-60 4 married 0
0.1     code= 41-50 4 married 0
0.04     code= 20-30 5 married 0
0.06     code= 31-40 5 married 0
0.0     code= 51-60 5 married 0
0.06     code= 41-50 5 married 0
0.0     code= 60+ 5 married 0
0.18     code= 20-30 1 single 0
0.17     code= 31-40 1 single 0
0.19     code= 51-60 1 single 0
0.2     code= 41-50 1 single 0
0.4     code= 60+ 1 single 0
0.23     code= 20-30 2 single 0
0.17     code= 31-40 2 single 0
0.

In [37]:
default[(default['Range_age']=='60+') & (default['MARRIAGE']=='single') & (default['EDUCATION']==2) & (default['SEX']==0)]


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,Range_age,ORD_AGE
6083,80000.0,0,2,single,64,2,2,0,0,0,...,74138.0,390.0,4000.0,2671.0,4000.0,2670.0,2923.0,1,60+,5
17743,80000.0,0,2,single,61,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,60+,5


In [38]:
# worse profile
for g in [0,1]:
    for m in ['married','single','others']:
        for e in [1,2,3,4,5]:
            for a in range_age:
                if len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                 (default['SEX']==g)]) > 100:
                    try:
                        print (round(
                            len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                (default['SEX']==g) & (default['default.payment.next.month']==1)]) /
                            len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                 (default['SEX']==g)]),2), '    code=',a,e,m,g)
                    except:
                        continue

0.21     code= 20-30 1 married 0
0.18     code= 31-40 1 married 0
0.21     code= 51-60 1 married 0
0.17     code= 41-50 1 married 0
0.27     code= 20-30 2 married 0
0.21     code= 31-40 2 married 0
0.27     code= 51-60 2 married 0
0.23     code= 41-50 2 married 0
0.27     code= 20-30 3 married 0
0.22     code= 31-40 3 married 0
0.24     code= 51-60 3 married 0
0.27     code= 41-50 3 married 0
0.18     code= 20-30 1 single 0
0.17     code= 31-40 1 single 0
0.2     code= 41-50 1 single 0
0.23     code= 20-30 2 single 0
0.17     code= 31-40 2 single 0
0.2     code= 41-50 2 single 0
0.23     code= 20-30 3 single 0
0.17     code= 31-40 3 single 0
0.19     code= 51-60 3 single 0
0.26     code= 41-50 3 single 0
0.23     code= 31-40 1 married 1
0.27     code= 51-60 1 married 1
0.22     code= 41-50 1 married 1
0.28     code= 20-30 2 married 1
0.26     code= 31-40 2 married 1
0.31     code= 51-60 2 married 1
0.26     code= 41-50 2 married 1
0.28     code= 31-40 3 married 1
0.3     code= 51-60 3 

In [39]:
# 0.31     code= 51-60 University married male

In [99]:
for g in [0,1]:
    for m in ['married','single','others']:
        for e in [1,2,3,4,5]:
            for a in range_age:
                if len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                 (default['SEX']==g)]) > 100:
                    try:
                        print (round(
                            len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                (default['SEX']==g) & (default['default.payment.next.month']==0)]) /
                            len(default[(default['Range_age']==a) & (default['MARRIAGE']==m) & (default['EDUCATION']==e) & 
                                 (default['SEX']==g)]),2), '    code=',a,e,m,g)
                    except:
                        continue

0.79     code= 20-30 1 married 0
0.82     code= 31-40 1 married 0
0.79     code= 51-60 1 married 0
0.83     code= 41-50 1 married 0
0.73     code= 20-30 2 married 0
0.79     code= 31-40 2 married 0
0.73     code= 51-60 2 married 0
0.77     code= 41-50 2 married 0
0.73     code= 20-30 3 married 0
0.78     code= 31-40 3 married 0
0.76     code= 51-60 3 married 0
0.73     code= 41-50 3 married 0
0.82     code= 20-30 1 single 0
0.83     code= 31-40 1 single 0
0.8     code= 41-50 1 single 0
0.77     code= 20-30 2 single 0
0.83     code= 31-40 2 single 0
0.8     code= 41-50 2 single 0
0.77     code= 20-30 3 single 0
0.83     code= 31-40 3 single 0
0.81     code= 51-60 3 single 0
0.74     code= 41-50 3 single 0
0.77     code= 31-40 1 married 1
0.73     code= 51-60 1 married 1
0.78     code= 41-50 1 married 1
0.72     code= 20-30 2 married 1
0.74     code= 31-40 2 married 1
0.69     code= 51-60 2 married 1
0.74     code= 41-50 2 married 1
0.72     code= 31-40 3 married 1
0.7     code= 51-60 3 

In [100]:
# 0.83     code= 31-40 2 single Female
# 0.83     code= 31-40 1 single Female

## Hypothesis Testing

In [54]:
default[((default['EDUCATION']==1) | (default['EDUCATION']==2))]

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,Range_age,ORD_AGE
0,20000.0,0,2,married,24,2,2,0,0,0,...,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,20-30,1
1,120000.0,0,2,single,26,0,2,0,0,0,...,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,20-30,1
2,90000.0,0,2,single,34,0,0,0,0,0,...,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,31-40,2
3,50000.0,0,2,married,37,0,0,0,0,0,...,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,31-40,2
4,50000.0,1,2,married,57,0,0,0,0,0,...,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,51-60,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29991,210000.0,1,2,married,34,3,2,2,2,2,...,2500.0,0.0,0.0,0.0,0.0,0.0,0.0,1,31-40,2
29993,100000.0,1,1,single,38,0,0,0,0,0,...,55004.0,2000.0,111784.0,4000.0,3000.0,2000.0,2000.0,0,31-40,2
29994,80000.0,1,2,single,34,2,2,2,2,2,...,81158.0,7000.0,3500.0,0.0,7000.0,0.0,4000.0,1,31-40,2
29997,30000.0,1,2,single,37,4,3,2,0,0,...,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1,31-40,2


In [84]:
# Education levels

pro_sup_ed = (len(default[(((default['EDUCATION']==1) | (default['EDUCATION']==2)) & (default['default.payment.next.month']==1))]))/ (len(default[((default['EDUCATION']==1) | (default['EDUCATION']==2))]))
pro_inf_ed = (len(default[(((default['EDUCATION']!=1) | (default['EDUCATION']!=2)) & (default['default.payment.next.month']==1))]))/ (len(default[((default['EDUCATION']!=1) | (default['EDUCATION']!=2))]))
print(pro_sup_ed,pro_inf_ed)

0.2179971562055657 0.2212


In [85]:
# Education levels
   
sample_default_ed, sample_size_ed = (len(default[(((default['EDUCATION']==1) | (default['EDUCATION']==2)) & (default['default.payment.next.month']==1))])), (len(default[((default['EDUCATION']==1) | (default['EDUCATION']==2))]))
sample_default_ned, sample_size_ned = (len(default[(((default['EDUCATION']!=1) | (default['EDUCATION']!=2)) & (default['default.payment.next.month']==1))])), (len(default[((default['EDUCATION']!=1) | (default['EDUCATION']!=2))]))

defaults_ed = np.array([sample_default_ed, sample_default_ned])
samples_ed = np.array([sample_size_ed, sample_size_ned])

stat, pvalue = proportions_ztest(defaults_ed,samples_ed)
pvalue

0.3684379574004112

In [98]:
def_male = (len(default[((default['SEX']==1) & (default['default.payment.next.month']==1))]))/ (len(default[((default['SEX']==1))]))
def_female = (len(default[((default['SEX']!=1) & (default['default.payment.next.month']==1))]))/ (len(default[((default['SEX']!=1))]))
print(def_male, def_female)

0.2416722745625841 0.20776280918727916


In [72]:
# Gender - 'Female = 0'

sample_default_male, sample_size_male = (len(default[((default['SEX']==1) & (default['default.payment.next.month']==1))])), (len(default[((default['SEX']==1))]))
sample_default_female, sample_size_female = (len(default[((default['SEX']!=1) & (default['default.payment.next.month']==1))])), (len(default[((default['SEX']!=1))]))

defaults_g = np.array([sample_default_male, sample_default_female])
samples_g = np.array([sample_size_male, sample_size_female])

stat, pvalue = proportions_ztest(defaults_g,samples_g)
pvalue

4.472804335814144e-12

In [95]:
# Credit limit
inf = (len(default[((default['LIMIT_BAL']<140000) & (default['default.payment.next.month']==1))])) / (len(default[((default['LIMIT_BAL']<140000))]))
sup = (len(default[((default['LIMIT_BAL']>140000) & (default['default.payment.next.month']==1))])) / (len(default[((default['LIMIT_BAL']>140000))]))

print(inf, sup)
                                                                                                  

0.28553744584278934 0.15730795377294357


In [96]:
# Credit limit

sample_default_inf, sample_size_inf = (len(default[((default['LIMIT_BAL']<140000) & (default['default.payment.next.month']==1))])), (len(default[((default['LIMIT_BAL']<140000))]))
sample_default_sup, sample_size_sup = (len(default[((default['LIMIT_BAL']>140000) & (default['default.payment.next.month']==1))])), (len(default[((default['LIMIT_BAL']>140000))]))

defaults_cl = np.array([sample_default_inf, sample_default_sup])
samples_cl = np.array([sample_size_inf, sample_size_sup])

stat, pvalue = proportions_ztest(defaults_cl,samples_cl)
pvalue

7.023005373133739e-154

In [ ]:
# sample_success_med, sample_size_med = (med_shared, 100)
# sample_success_sca, sample_size_sca = (sca_shared, 100)
# successes = np.array([sample_success_med, sample_success_sca])
# samples = np.array([sample_size_med, sample_size_sca])
# stat, pvalue = proportions_ztest(successes,samples)
# pvalue